In [ ]:
from positional_encodings.torch_encodings import PositionalEncoding1D, PositionalEncoding2D, PositionalEncoding3D, Summer
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy.io import savemat, loadmat
import random
random.seed(2)
torch.manual_seed(2)
torch.cuda.manual_seed(2)
np.random.seed(2)

mdict = loadmat("PosEncode.mat")
## TODO: this cell should be saves seperatly and the result could be loaded at the initially
PosEncoding = mdict["PositionalEncoding"]

## Load Postional Condition, Repeat it based on batch_size and Convert it Torch.Tensor
# PosEncoding = scipy.loatmat()
PosEncoding = torch.tensor(PosEncoding)
# PosEncoding = PosEncoding[np.newaxis,:,:,:].repeat(batch_size,1,1,1)
print(PosEncoding.shape)


In [ ]:
import matplotlib.pyplot as plt
import scipy.fftpack
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms, utils
import torchvision
import torch
import glob
from medpy.io import load, save
import sys
from scipy.io import savemat

save_to_dir = 'J:/Anvari/MsThesis/K-Space-Net_2sliceApart/T2/40_Percent/Gaussian/'
# save_to_dir = 'J:/Anvari/MsThesis/K-Space-Net_2sliceApart/T2/half_slices/'
model_dir = 'J:/Anvari/MsThesis/K-Space-Net_2sliceApart/T2_model/'
model_dir2 = 'J:/Anvari/MsThesis/K-Space-Net_2sliceApart/T1_model/'
LINE_CLEAR = '\x1b[2K' # <-- ANSI sequence
train_root = 'J:\\Anvari\\proccessed_data_257\\train\\'
validation_root = 'J:\\Anvari\\proccessed_data_257_2sliceApart\\validation\\'
test_root = 'J:\\Anvari\\proccessed_data_257_2sliceApart\\test\\'

transform_2015 = transforms.Compose([transforms.ConvertImageDtype(torch.float),
                                     transforms.Normalize(0.5, 0.5, 0.5)])


def imshow(img, batch_size, name, cmap):
    if torch.is_tensor(img):
        npimg = img.numpy()
    else:
        npimg = img
    if npimg.ndim > 2:
        npimg = np.transpose(npimg, (1, 2, 0))
    plt.figure(figsize=(7, 4))
    
    if cmap != 'gray':
        # c = 1 / np.log(1 + np.max(npimg[:,:,0]))
        # image = c * (np.log(npimg[:,:,0] + 1))
        if npimg.ndim > 2:
            image = npimg[:,:,0]
    else:
        if npimg.ndim > 2:
            image = npimg[:,:,0]
        else:
            image = npimg

    if cmap != 'gray':
        image[-1,-1] = 0.1
        image[-2,-1] = 0
        plt.imshow(image,interpolation = 'none',cmap=cmap,vmin=0,vmax=0.1)
        plt.colorbar(orientation="horizontal")
        plt.axis('off')
        plt.savefig(save_to_dir+'{}_01.png'.format(name), format='png', dpi=600)
        plt.figure(figsize=(7, 4))
        image[-1,-1] = 1
        image[-2,-1] = 0
        plt.imshow(image,interpolation = 'none',cmap=cmap,vmin=0,vmax=1)
        plt.colorbar(orientation="horizontal")
        plt.axis('off')
        plt.savefig(save_to_dir+'{}_10.png'.format(name), format='png', dpi=600)
    else:
        image[-1,-1] = 1
        image[-2,-1] = 0
        plt.imshow(image,interpolation = 'none',cmap=cmap,vmin=0,vmax=1)
        plt.axis('off')
        plt.savefig(save_to_dir+'{}.png'.format(name), format='png', dpi=600)

def load_mask(mask_name):
    dir = 'C:\\Users\\mehrdad\\MS_projects\\mask\\Build mask'
    matfile = scipy.io.loadmat(dir + '\\{}.mat'.format(mask_name)) 
    print(type(matfile))
    print(matfile.keys())
    # print(matfile.get("__header__"))
    mask = np.zeros((6,257,257))
    mask_P = matfile.get('PrimaryMask')
    mask_S = matfile.get('SecondaryMask')
    mask_P = np.logical_or(mask_P,np.flip(mask_P))
    mask_S = np.logical_or(mask_S,np.flip(mask_S))
    mask[0,:,:]=mask_S
    mask[1,:,:]=mask_P
    mask[2,:,:]=mask_S
    mask[3,:,:]=mask_P
    mask[4,:,:]=mask_S
    mask[5,:,:]=mask_P
    # mask[0,:,:]=1
    # mask[1,:,:]=1
    # mask[2,:,:]=1
    # mask[3,:,:]=1
    # mask[4,:,:]=0
    # mask[5,:,:]=1
    # print(mask.sum() / (256 * 256))
    print(mask.shape)
    print(mask.dtype)
    print(type(mask))
    plt.imshow(mask_P)
    plt.title(mask_name)
    plt.axis('off')
    plt.show()
    plt.imshow(mask_S)
    plt.title(mask_name)
    plt.axis('off')
    plt.show()
    plt.imshow(np.logical_or(mask_P,mask_S))
    plt.title(mask_name)
    plt.axis('off')
    plt.show()
    imshow(mask_P, 10, 'primary', cmap='gray')
    imshow(mask_S, 10, 'secodary', cmap='gray')
    imshow(np.logical_or(mask_P,mask_S), 10, 'Union', cmap='gray')

    return mask

# mask = load_mask('Deterministic1DMask_40_257')
mask = load_mask('GaussianDistribution1DMask_40_257')
# mask = load_mask('UniformDistribution1DMask_10_257')
                
mask_c = 1 - mask

# def cal_fft_for_all_channels(x):
#     fft = np.zeros_like(x).astype('complex128')
#     for i in range(6):
#         fft[i, :, :] = scipy.fftpack.fft2(x[i, :, :])
#         fft[i, :, :] = scipy.fftpack.fftshift(fft[i, :, :])
#     return np.real(fft).astype('float32'),np.imag(fft).astype('float32')

def cal_fft_for_all_channels(x):
    # fft = torch.zernp.zeros_like(x).astype('complex128'), dtype=torch.complex64
    fft = torch.zeros_like(x,dtype=torch.complex64)
    fft = torch.fft.fft2(x)
    fft = torch.fft.fftshift(fft,dim=(1,2))
    # for i in range(6):
    #     fft[i, :, :] = scipy.fftpack.fft2(x[i, :, :])
    #     fft[i, :, :] = scipy.fftpack.fftshift(fft[i, :, :])
    return torch.real(fft).to(torch.float32) ,torch.imag(fft).to(torch.float32)

def to_bad_img(x, mask):
    x = (x + 1.) / 2.
    
#     print(f"mask shape is:{x.shape}")
    for i in range(6):
        fft = scipy.fftpack.fft2(x[i, :, :])
        fft = scipy.fftpack.fftshift(fft)
        fft = fft * mask[i,:,:]
        fft = scipy.fftpack.ifftshift(fft)
        x[i, :, :] = scipy.fftpack.ifft2(fft)
        x[i, :, :] = np.abs(x[i, :, :])
        x[i, :, :] = x[i, :, :] * 2 - 1
    return x

class Brats2013_2D(Dataset):
    def __init__(self, root, transform=transform_2015 , PE = PosEncoding):
        self.img_dir = root
        self.positionalEncoding = PE
        self.transform = transform
        file_list = glob.glob(root + "*.mha")
        self.data = []
        for file_path in file_list:
            layer_name = file_path.split("\\")[-1].split(".")[-2].split("_")[-1]
            self.data.append([file_path, int(layer_name)])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, LayerNum = self.data[idx]
        image, image_header = load(img_path)
        image = torch.tensor(image).cuda().to(torch.float32)
        image_fft_r,image_fft_i = cal_fft_for_all_channels(image)
        # image_fft_r = torch.tensor(image_fft_r)
        # image_fft_i = torch.tensor(image_fft_i)
        # image = torch.tensor(image)
        return image_fft_r, image_fft_i, image, torch.tensor(LayerNum), torch.tensor(self.positionalEncoding)




def imsave(img,epoch,i,name,batch_size):
    img_slices = np.zeros((6*batch_size,240,240))
    for j in range(batch_size*6):
        img_slices[j,:,:] = img[int(j//6),int(j%6),:,:]
    save(img_slices,save_to_dir+f'{name}_{epoch}_{i}.mha')

batch_size = 10

dataset = Brats2013_2D(root=train_root , PE = PosEncoding)
dataset_val = Brats2013_2D(root=validation_root, PE=PosEncoding)
dataset_test = Brats2013_2D(root=test_root, PE=PosEncoding)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
img_fft_r, img_fft_i, img ,layer , PosEn = next(iter(dataloader))
print(type(img_fft_r))
print(img_fft_r.dtype)
print('img_fft_r shape = {}'.format(img_fft_r.shape))
print('layer = {}'.format(layer.shape))
print('PosEn shape = {}'.format(PosEn.shape))
plt.imshow(img_fft_r[0,0,:,:].cpu())
plt.show()

In [ ]:
from torch import nn
import math
class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, layer):
        device = layer.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = layer[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings


In [ ]:
import torch.nn.functional as F
import torch
from torch import nn
import numpy as np
from torchsummary import summary
from torchvision import models
from einops import rearrange
if torch.cuda.is_available():
    print('it works')
else:
    print('it doesnt work')
device = torch.device("cuda:0" if (torch.cuda.is_available() and 1 > 0) else "cpu")
print('device is {}'.format(device))


class KspaceNetT1(nn.Module):
    def __init__(self,ngpu):
        super(KspaceNetT1, self).__init__()
        self.df_dim = 32     ## TODO: normalizing the input k-space
        self.posEn_dim = 34
        self.layEn_dim = 32
        self.conv1 = nn.Conv2d(10 + self.posEn_dim + self.layEn_dim , 4*self.df_dim, (1, 1), padding=(0, 0))
        self.conv2 = nn.Conv2d(4*self.df_dim, 2*self.df_dim, (1, 1), padding=(0, 0))
        self.conv3 = nn.Conv2d(2*self.df_dim, 1*self.df_dim, (1, 1), padding=(0, 0))
        self.conv4 = nn.Conv2d(1*self.df_dim, 2, (1, 1), padding=(0, 0))

        self.batch1 = nn.BatchNorm2d(4 * self.df_dim)
        self.batch2 = nn.BatchNorm2d(2 * self.df_dim)
        self.batch3 = nn.BatchNorm2d(1 * self.df_dim)

        ## TODO: an MLP model for Layer encoding 
        self.layer_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(self.layEn_dim),
            nn.Linear(self.layEn_dim, self.layEn_dim),
            nn.GELU(),
            nn.Linear(self.layEn_dim, self.layEn_dim),
        )
        
        self.layer_mlp2 = nn.Sequential(
            SinusoidalPositionEmbeddings(self.layEn_dim),
            nn.Linear(self.layEn_dim, self.layEn_dim),
            nn.GELU(),
            nn.Linear(self.layEn_dim, 2*self.posEn_dim),
        )
        
        self.layer_mlp3 = nn.Sequential(
            SinusoidalPositionEmbeddings(self.layEn_dim),
            nn.Linear(self.layEn_dim, self.layEn_dim),
            nn.GELU(),
            nn.Linear(self.layEn_dim, 10),
        )
        
        self.convLEPEforX = nn.Conv2d(self.posEn_dim + 10, 10*2, (1, 1), padding=(0, 0))
        
    def forward(self, x, PE, LE):

        layerEmbeddingLatent = self.layer_mlp(LE)
        
        LEchs = rearrange(layerEmbeddingLatent, "b c -> b c 1 1")
        LEchs = LEchs.repeat(1,1,257,257) ## TODO: check if scale and shifting X using LEchs is any better
        ## TODO: Check if scale and shifting PE using LEch is any better
        LEforPE = self.layer_mlp2(LE)
        LEforPE2 = rearrange(LEforPE, "b c -> b c 1 1")
        scale_shift = LEforPE2.chunk(2, dim=1)
        scale, shift = scale_shift
        PE = PE * (scale + 1) + shift
        ## TODO: Check if scale and shiftinf PE using LEch and concat it to X while the X itself is scaled and shifted by scaled and shifted PE 
        LEforX = self.layer_mlp3(LE)
        LEforX2 = rearrange(LEforX, "b c -> b c 1 1")
        LEforX2 = LEforX2.repeat(1,1,257,257)
        LEPEforX = torch.concat((PE, LEforX2),dim=1)
        LEPEforX2 = self.convLEPEforX(LEPEforX)
        scale_shift = LEPEforX2.chunk(2, dim=1)
        scale, shift = scale_shift
        x = x * (scale + 1) + shift
        ## TODO: Concatinate input (x), Positional Encoding (PE), Layer Encoding (LE)
        
        
        x = torch.concat((x, PE, LEchs),dim=1)

        x = self.conv1(x)
        x = nn.LeakyReLU(0.2)(x)
        x = self.batch1(x)

        x = self.conv2(x)
        x = nn.LeakyReLU(0.2)(x)
        x = self.batch2(x)

        x = self.conv3(x)
        x = nn.LeakyReLU(0.2)(x)
        x = self.batch3(x)

        x = self.conv4(x)
#         x = nn.Tanh()(x)
        
        return x
    

In [ ]:
from tqdm import tqdm
import torch.optim as optim
import torchvision.utils as vutils
from torchmetrics import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio
import os
import json


ngpu = 1
lr = 0.001
lr_decay = 0.5
decay_every = 5
n_epoch = 9999
beta1 = 0.5
batch_size = 10
workers = 1

## check if cuda is available or not?
if torch.cuda.is_available():
    print('CUDA is Available!')
else:
    print('CUDA is NOT Available!')
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print('device is {}'.format(device))

## Empety torch cuda cache
torch.cuda.empty_cache()

## Define the model
model = KspaceNetT1(ngpu).to(device)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))


## Define optimizer
print('Setup Adam optimizers for D')
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(beta1, 0.999))
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)

## Define Loss
criterion = torch.nn.MSELoss()

## Define Metrics
psnr = PeakSignalNoiseRatio()
ssim = StructuralSimilarityIndexMeasure(data_range=1.0)

## Load the model if exists, pre-trained model

# if  os.path.isfile(model_dir + 'modelt2.pt'):
#     model = torch.load(model_dir + 'modelt2.pt')
    
# model.eval()

if  os.path.isfile(model_dir + 'modelt1.pt'):
    model = torch.load(model_dir + 'modelt1.pt')
    
model.eval()
    
    
print(f'scheduler : {scheduler.get_last_lr()}')



## Define some List to Keep track of the Learning 

#  Validation Data
PSNR_V = []
SSIM_V = []
MSE_IMG_V = []


PSNR_V2 = []
SSIM_V2 = []
MSE_IMG_V2 = []


mask = torch.tensor(mask).to(device)
mask_c = torch.tensor(mask_c).to(device)


t_p = 4
t_c = 1
# mask[t_p,:,:] = 0
# mask[t_c,:,:] = 1
# mask[0,:,:] = 1
# mask[2,:,:] = 1
# mask[3,:,:] = 1
# mask[5,:,:] = 1
## show the predited image
with torch.no_grad():
            
        PSNR_Vt = 0
        SSIM_Vt = 0
        MSE_FFT_Vt = 0
        MSE_IMG_Vt = 0
        MSE_RealEven_Vt = 0
        MSE_ImagOdd_Vt  = 0
        
        PSNR_Vt2 = 0
        SSIM_Vt2 = 0
        MSE_FFT_Vt2 = 0
        MSE_IMG_Vt2 = 0
        MSE_RealEven_Vt2 = 0
        MSE_ImagOdd_Vt2  = 0

        print('Calculating loss and metrics for Validation dataset ... ')
        for l, data_val in tqdm(enumerate(dataloader_test, 0)):
            # load one batch of data

            image_fft_r, image_fft_i, image, LayerNum, PosEncoding = data_val
            for i in range(6):
                image_fft_r[:,i,:,:] = image_fft_r[:,i,:,:].to(device) * mask[i,:,:]
                image_fft_i[:,i,:,:] = image_fft_i[:,i,:,:].to(device) * mask[i,:,:]

            if t_p == 4:  
                ListConChIdxs = [0,t_c,2,3,5] ## index 1 is the T1 middle layer that should be predicted
            if t_p == 1:
                ListConChIdxs = [0,2,3,t_c,5] ## index 1 is the T1 middle layer that should be predicted

            image_fft_in = torch.concat((image_fft_r[:,ListConChIdxs,:,:],image_fft_i[:,ListConChIdxs,:,:]), dim = 1).to(device)
            image_fft_out = torch.concat((image_fft_r[:,t_p,:,:][:,np.newaxis,:,:],image_fft_i[:,t_p,:,:][:,np.newaxis,:,:]), dim = 1).to(device)
            
            PosEncoding = PosEncoding.to(device)
            ## Inference
            Predition = model(image_fft_in.float(),PosEncoding.float(),LayerNum.to(device))
            
            # correct values of estimated image using true ones
            Predition[:,0,:,:] = Predition[:,0,:,:] * mask_c[t_p,:,:] + image_fft_r[:,t_p,:,:].to(device) * mask[t_p,:,:]
            Predition[:,1,:,:] = Predition[:,1,:,:] * mask_c[t_p,:,:] + image_fft_i[:,t_p,:,:].to(device) * mask[t_p,:,:]
            
            ## Calculate the Loss term
            complexPrediction  = Predition[:,0,:,:][:,np.newaxis,:,:]+Predition[:,1,:,:][:,np.newaxis,:,:]*1j
            complexPrediction = torch.fft.ifftshift(complexPrediction,dim=(2,3))
            image_Predition = torch.fft.ifft2(complexPrediction)
            image_Predition = torch.real(torch.tensor(image_Predition).float())
            for k in range(image_Predition.shape[0]):
                nume = torch.sqrt(torch.sum(torch.square((image_Predition[k,:,:,:][np.newaxis,:,:,:]-image[k,t_p,:,:][np.newaxis,np.newaxis,:,:].to(device)))))
                deno = torch.sqrt(torch.sum(torch.square(image[k,t_p,:,:][np.newaxis,np.newaxis,:,:])))
                Loss_img = nume/deno
                psnrValue = psnr(image_Predition[k,:,:,:][np.newaxis,:,:,:].cpu(), image[k,t_p,:,:][np.newaxis,np.newaxis,:,:].cpu())
                ssimValue = ssim(image_Predition[k,:,:,:][np.newaxis,:,:,:].cpu(), image[k,t_p,:,:][np.newaxis,np.newaxis,:,:].cpu())
                PSNR_V.append(psnrValue.item())
                SSIM_V.append(ssimValue.item())
                MSE_IMG_V.append(Loss_img.item())

            if l==1:
                imshow(vutils.make_grid(image_Predition.cpu(), padding=2, nrow=5, normalize=False), batch_size, 'fix_mask',cmap='gray')
                imshow(vutils.make_grid(image[:,t_p,:,:][:,np.newaxis,:,:].cpu(), padding=2, nrow=5, normalize=False), batch_size, 'GroundTruth_mask',cmap='gray')
                imshow(vutils.make_grid(torch.abs(image[:,t_p,:,:][:,np.newaxis,:,:].cpu()-image_Predition.cpu()), padding=2, nrow=5, normalize=False), batch_size, 'diff_pred_GT',cmap='jet')

            
            ##########################################################3
            ## correct values of estimated image using true ones
            Predition[:,0,:,:] = image_fft_r[:,t_p,:,:].to(device) * mask[t_p,:,:]
            Predition[:,1,:,:] = image_fft_i[:,t_p,:,:].to(device) * mask[t_p,:,:]
            
            ## Calculate the Loss term
            complexPrediction  = Predition[:,0,:,:][:,np.newaxis,:,:]+Predition[:,1,:,:][:,np.newaxis,:,:]*1j
            complexPrediction = torch.fft.ifftshift(complexPrediction,dim=(2,3))
            image_Predition = torch.fft.ifft2(complexPrediction)
            image_Predition = torch.real(torch.tensor(image_Predition).float())

            for k in range(image_Predition.shape[0]):
                nume = torch.sqrt(torch.sum(torch.square((image_Predition[k,:,:,:][np.newaxis,:,:,:]-image[k,t_p,:,:][np.newaxis,np.newaxis,:,:].to(device)))))
                deno = torch.sqrt(torch.sum(torch.square(image[k,t_p,:,:][np.newaxis,np.newaxis,:,:])))
                Loss_img = nume/deno
                psnrValue = psnr(image_Predition[k,:,:,:][np.newaxis,:,:,:].cpu(), image[k,t_p,:,:][np.newaxis,np.newaxis,:,:].cpu())
                ssimValue = ssim(image_Predition[k,:,:,:][np.newaxis,:,:,:].cpu(), image[k,t_p,:,:][np.newaxis,np.newaxis,:,:].cpu())
                PSNR_V2.append(psnrValue.item())
                SSIM_V2.append(ssimValue.item())
                MSE_IMG_V2.append(Loss_img.item())

            if l==1:
                imshow(vutils.make_grid(image_Predition.cpu(), padding=2, nrow=5, normalize=False), batch_size, 'fix_zerofill',cmap='gray')
                imshow(vutils.make_grid(torch.abs(image[:,t_p,:,:][:,np.newaxis,:,:]-image_Predition).cpu(), padding=2, nrow=5, normalize=False), batch_size, 'diff_zerofill_GT',cmap='jet')

                # imshow(vutils.make_grid(image[:,t_p,:,:][:,np.newaxis,:,:].cpu(), padding=2, nrow=5, normalize=True), batch_size, 'GroundTruth_zerofill')


        mdic2 = {"PSNR_ZF": PSNR_V2,"SSIM_ZF": SSIM_V2,"NMSE_ZF": MSE_IMG_V2,"PSNR_M": PSNR_V,"SSIM_M": SSIM_V,"NMSE_M": MSE_IMG_V}
        # print('LOG:')
        # print(mdic2)
        savemat(save_to_dir+"LOG.mat", mdic2)
        with open(save_to_dir+"LOG.json", "w") as outfile:
            json.dump(mdic2, outfile)